# Threat Actor Assessment
This notebook leverages the threat actor report summaries to assess their threat using the threat box model. Prioritizing threat actors is important to put your resources to best use. However, threat actor assessment can be very subjective. By assessing threat actors using a model, we aim to generate consistent results that limit the bias of the analyst. 

This notebook provides example usage of the prompt and some other variants to generate more accurate results.

References:
- https://www.sans.org/white-papers/39585/

# Improvements ideas
- [x] Optimize prompt using LLM
- [ ] Look for methods to better assess intent, since the model tends to select '3 - sector association' too frequently, while the victim sector does match exactly.
- [ ] Look for methods to better assess capability, since the context about sources and amount of evidence is missing the threat actor report summaries. 

# Import modules

In [51]:
from langchain_openai import ChatOpenAI
from langchain.document_loaders import UnstructuredMarkdownLoader
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
import openai
import os
import glob

# Define initial variables and OpenAI API key

In [44]:
current_directory = os.path.dirname("__file__")
knowledge_directory = os.path.join(current_directory, "knowledge")
documents_directory = os.path.join(knowledge_directory, "documents")
summaries_directory = os.path.join(documents_directory, "summaries")


load_dotenv()
# Get your key: https://platform.openai.com/account/api-keys
openai.api_key = os.getenv("OPENAI_API_KEY")

# Setup prompt
See end of notebook for prompt experimentation

In [61]:
prompt = PromptTemplate.from_template("""
# Cyber Threat Evaluation Task

## Objective

Your task is to assess a threat actor's potential to target a specific victim using the provided cyber threat intelligence reports. Employ the Threat Box Model for a structured analysis, focusing on the actor's capability and intent. It's imperative to base your evaluation strictly on the information within the reports, prioritizing accuracy and avoiding unsupported assumptions. Clarity, transparency, and a methodical approach are key. When in doubt, opt for a conservative estimate over an inaccurately high threat level.

## Victim Profile

- **Company:** Undefined
- **Industry:** Manufacturing
- **Location:** Europe

## Evaluation Criteria

### A. Intent and Willingness

Evaluate the threat actor's intent and willingness to target the specified victim, using the following scale:

- **5:** Target-Specific Data - Intent is based on unique objectives achievable only within the victim's network.
- **4:** Sector + Regional Association - Intent due to the victim's industry and regional context.
- **3:** Sector Association - Intent due to the victim's industry affiliation.
- **2:** Regional Association - Intent based on the victim's geographical operations.
- **1:** Target of Opportunity - Non-specific intent; the victim is an opportunistic target.

#### Willingness Modifier

Adjust the intent score based on potential constraints affecting the actor's willingness:

- **-0:** No impact - Strained diplomatic relations or perceived significant economic disruption by the threat actor from the victim's operations.
- **-1:** Moderate impact - Moderate diplomatic and economic interactions between the threat actor and the EU.
- **-2:** Strong impact - Strong diplomatic, economic, and security ties with the EU.

### B. Capabilities and Novelty

Assess the threat actor's capabilities and the novelty of their methods:

- **5:** Significant Capability - Strong evidence from multiple sources of past similar activities.
- **4:** Credible Capability - Reliable evidence of the capability, moderately confirmed.
- **3:** Limited Capability - Some evidence, though from limited sources.
- **2:** Possible Capability - Very limited evidence; attack feasibility confirmed.
- **1:** Not Capable - No evidence of capability; feasibility unconfirmed.

#### Novelty Modifier

Adjust the capability score based on the sophistication of the actor's methods:

- **-0:** Custom toolset for each campaign, showing advanced skills.
- **-1:** Limited/high-cost toolset used across campaigns.
- **-2:** Widely available toolset.

## Task Instructions

1. **Review Reports:** Analyze the provided cyber threat reports within the context of the Threat Box Model.
2. **Evaluate Intent and Capability:** Determine the threat actor's intent and capability scores based on the criteria above.
3. **Justify Your Ratings:** Clearly explain the rationale behind each score, ensuring your logic is transparent.
4. **Compile a Matrix:** Summarize your findings in a matrix, listing scores for each dimension, including modifiers.

## Final Deliverable

Submit your analysis, including the justification for each score and the final matrix. Your evaluation should reflect a thorough understanding of the threat actor's potential actions against the specified victim, grounded in the provided intelligence reports.

## Threat Actor Input

{context}

"""
)

# Setup LLM

In [63]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=openai.api_key)
chain = prompt | llm

# Invoke request
Used 'Achilles' as example threat actor

In [64]:
example_report = os.path.join(summaries_directory, "Achilles_summary.md")
loader = UnstructuredMarkdownLoader(example_report)
report = loader.load()
chain.invoke(({"context": report[0].page_content}))


AIMessage(content="## Threat Actor Evaluation\n\n### A. Intent and Willingness\n\nBased on the provided reports, the threat actor known as Achilles demonstrates a clear intent to target high-value corporate internal networks, including those of multinational companies and government organizations. Their focus on private companies and government entities in the British Commonwealth, as well as their claimed access to networks like UNICEF, indicates a specific intent to target organizations within these sectors. Therefore, the intent score is:\n\nIntent Score: 4 (Sector + Regional Association)\n\nGiven the threat actor's ties to Iranian hackers and the potential connection to Iran based on activity following Iranian time zones, there may be some impact on their willingness to target a victim in Europe due to diplomatic and economic interactions. Therefore, the willingness modifier is:\n\nWillingness Modifier: -1 (Moderate impact)\n\nAdjusted Intent Score: 3 (Sector Association)\n\n### B.

# Prompt experimentation
We are not yet satisfied with the quality of the output. It is lacking accuracy and consistency. Below are are experimenting with some other prompt versions

# 1. Default prompt

# Cyber Threat Evaluation Task

## Objective

Your task is to assess a threat actor's potential to target a specific victim using the provided cyber threat intelligence reports. Employ the Threat Box Model for a structured analysis, focusing on the actor's capability and intent. It's imperative to base your evaluation strictly on the information within the reports, prioritizing accuracy and avoiding unsupported assumptions. Clarity, transparency, and a methodical approach are key. When in doubt, opt for a conservative estimate over an inaccurately high threat level.

## Victim Profile

- **Industry:** High-Tech Manufacturing
- **Location:** Europe

## Evaluation Criteria

### A. Intent and Willingness

Evaluate the threat actor's intent and willingness to target the specified victim, using the following scale:

- **5:** Target-Specific Data - Intent is based on unique objectives achievable only within the victim's network.
- **4:** Sector + Regional Association - Intent due to the victim's industry and regional context.
- **3:** Sector Association - Intent due to the victim's industry affiliation.
- **2:** Regional Association - Intent based on the victim's geographical operations.
- **1:** Target of Opportunity - Non-specific intent; the victim is an opportunistic target.

#### Willingness Modifier

Adjust the intent score based on potential constraints affecting the actor's willingness:

- **-0:** No impact - Strained diplomatic relations or perceived significant economic disruption by the threat actor from the victim's operations.
- **-1:** Moderate impact - Moderate diplomatic and economic interactions between the threat actor and the EU.
- **-2:** Strong impact - Strong diplomatic, economic, and security ties with the EU.

### B. Capabilities and Novelty

Assess the threat actor's capabilities and the novelty of their methods:

- **5:** Significant Capability - Strong evidence from multiple sources of past similar activities.
- **4:** Credible Capability - Reliable evidence of the capability, moderately confirmed.
- **3:** Limited Capability - Some evidence, though from limited sources.
- **2:** Possible Capability - Very limited evidence; attack feasibility confirmed.
- **1:** Not Capable - No evidence of capability; feasibility unconfirmed.

#### Novelty Modifier

Adjust the capability score based on the sophistication of the actor's methods:

- **-0:** Custom toolset for each campaign, showing advanced skills.
- **-1:** Limited/high-cost toolset used across campaigns.
- **-2:** Widely available toolset.

## Task Instructions

1. **Review Reports:** Analyze the provided cyber threat reports within the context of the Threat Box Model.
2. **Evaluate Intent and Capability:** Determine the threat actor's intent and capability scores based on the criteria above.
3. **Justify Your Ratings:** Clearly explain the rationale behind each score, ensuring your logic is transparent.
4. **Compile a Matrix:** Summarize your findings in a matrix, listing scores for each dimension, including modifiers.

## Final Deliverable

Submit your analysis, including the justification for each score and the final matrix. Your evaluation should reflect a thorough understanding of the threat actor's potential actions against the specified victim, grounded in the provided intelligence reports.
```

-----------------------------------------------
# 2. Prompt improved through LLM
Let LLM redefine the prompt.

# Refined Cyber Threat Evaluation Task 

## Objective

Your task is to conduct a precise and consistent evaluation of a threat actor's potential to target a specific victim, based on cyber threat intelligence reports. Utilize the model described below for structured analysis, focusing exclusively on the actor's capability and intent as evidenced in the reports. Ensure your evaluation is grounded in factual information, avoiding extrapolation or assumptions. Aim for clarity, consistency, and a methodical approach in your analysis. When evidence is insufficient, default to a lower, more conservative rating.

## Victim Profile

- **Industry:** High-Tech Manufacturing
- **Region:** Western-Europe
- **Crown jewels:** Intellectual property

## Evaluation Criteria

### A. Intent and Willingness

Assess the threat actor's specific intent and willingness to target the victim. Only assign a rating if it is supported by direct evidence from the reports. Each rating must be substantiated by the criteria of the preceding levels:

- **5:** Target-Specific Data - Clear evidence of intent targeting the victim's unique assets or operations. Must directly relate to the victim's crown jewels.
- **4:** Ideology Association - Intent linked to ideological motivations directly associated with the victim.
- **3:** Sector Association - Intent due to direct industry match. Avoid associating unrelated sectors even if they share broad characteristics. The exact victim sector must be mentioned in the report evidence for it to be a match. Threat actors choose their sectors precisely, and loose connections don't make sense here. 
- **2:** Regional Association - Intent based on the victim's geographical presence.
- **1:** Target of Opportunity - General intent; the victim is considered an opportunistic target without specific alignment to the threat actor's known preferences.

#### Willingness Modifier

Modify the intent score based on geopolitical or economic factors that could influence the actor's willingness:

- **-0:** No impact - Applies to actors from countries with a history of cyber aggression against Western targets.
- **-1:** Moderate impact - For actors from countries with moderate interactions with the EU. Default score for aggressive nations is 0.
- **-2:** Strong impact - Applies to actors from countries with strong ties to the EU, potentially deterring aggressive actions.

### B. Capabilities and Novelty

Evaluate the threat actor's capabilities and the innovation of their methods, considering the recency of their activities:

- **5:** Significant Capability - Robust evidence from multiple sources of similar past activities.
- **4:** Credible Capability - Reliable evidence, moderately confirmed.
- **3:** Limited Capability - Evidence exists, though it's limited.
- **2:** Possible Capability - Minimal evidence; feasibility is speculative.
- **1:** Not Capable - No evidence of capability.

#### Novelty Modifier

Adjust based on the sophistication of the actor's methods:

- **-0:** Use of custom tools per campaign, indicating high skill.
- **-1:** Use of limited or costly tools across campaigns.
- **-2:** Reliance on widely available tools.

Absent specific evidence of tool sophistication, default to -2.

### Attack Types
We split threat actor attacks into four types. We want to provide a score for above dimensions for all four of them. We therefore want to see 4 matrices being outputted. If an attack type is not mentioned in the provided reports, rate it minimum for capability and intent since no evidence is provided. Do not make assumptions based on information provided about the other attack types for the threat actor. The attack types are:
-**Espionage:** Any attack of a threat actor designed to impact the confidentiality of data and/or information system (e.g., trade secrets, government data, etc.)
-**Destructive:** Any attack of a threat actor designed to permanently impact the integrity of data and/or information system. 
-**Disruptive:** Any attack intended to temporarily degrade the availability of data or information systems (e.g., DDoS, web defacements, etc)
-**Cyber-Crime:** Any attack intended for near-term financial profit (e.g., ransomware, point-of-sale (PoS) malware, etc). Intellectual Property (IP) theft that is not immediately sold is considered espionage rather than cyber-crime.  

## Task Instructions

1. **Review Reports:** Analyze the cyber threat reports, applying the model described above. Synthesize an overall score for the threat actor, rather than scoring per report.
2. **Evaluate Intent and Capability:** Determine the threat actor's intent and capability scores based on the evidence. Aggregate your findings from all reports. Be precise and exact; we'd rather have lower score output then to make assumptions or ambiguous links. Do this for all four attack types.
3. **Compile a Matrix:** Present your findings in a matrix, detailing scores for each dimension, including modifiers. Make this easy to parse by Python. 

## Final Deliverable

Provide a detailed analysis with justifications for each score and the final matrix. Your evaluation should accurately reflect the threat actor's potential actions against the specified victim, strictly based on the intelligence reports.

# Threat Actor Context Input
